### Requirements

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama_index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
%pip install llama-index-llms-huggingface

In [4]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Rag App with using LLama-index

In [ ]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

In [7]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="CohereForAI/aya-23-8B", #meta-llama/Llama-2-13b-chat-hf
    model_name="CohereForAI/aya-23-8B", #meta-llama/Llama-2-13b-chat-hf
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [55]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import ServiceContext


embed_model=HuggingFaceEmbedding(model_name="google-bert/bert-base-multilingual-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [56]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-56-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [ ]:
service_context

In [58]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [59]:
query_engine=index.as_query_engine()

In [70]:
response=query_engine.query("İsteğe bağlı stajlar kaç kez yapılır")

In [72]:
print("DÖKÜMAN:\n\n",response.source_nodes[0].text)
print("===="*40)
print("CEVAP:\n\n",response)

DÖKÜMAN:

 (2) Zorunlu ve isteğe bağlı staj, öğrencinin talep etmesi durumunda 10 iş gününden kısa 
olmamak koşulu ile ikiye bölünebilir.  
(3) Öğrencinin kayıtlı olduğu Bölüm Staj Komisyonun uygun görüşü ve bu yönergede 
belirtilen usul ve esaslara uygun olmak kaydıyla Fakülte Yönetim Kurulu kararı ile öğrenci, 
isteğe bağlı olarak veya öğrenim gördüğü program kapsamında belirlenen staj süresinden 
daha uzun süreli staj yapab ilir. Bu fıkra kapsamındaki fazlalık staj süreleri kredilendirilebilir 
ancak mezuniyet kredisi hesabına dâhil edilmez.  
(4) Stajların yarıyıl veya yaz tatiline rastlayan aylarda yapılması esastır. Resmi tatil günlerinde yapılan çalışmalar staj süresi olarak  kabul edilmez.  
(5) Staj, haftada üç iş gününden az olmamak şartıyla  eğitim ve öğretim dönemi, yaz 
okulu ve genel sınav dönemlerinde yapılabilir. Ancak, bu maddeden yararlanılabilmesi için haftalık ders programı ve/veya genel sınav takviminin ilgili Bölüm Başkanlığı tarafından ilan edilmiş olması gerekli

In [73]:
response=query_engine.query("Kınama cezasını gerektiren disiplin suçları nelerdir")

In [76]:
response

Response(response='\nKınama cezasını gerektiren disiplin suçları şunlardır:\n\nd) Sınavlarda kopyaya teşebbüs etmek.\n\ne) Sınavlarda kopya çekmek veya çektirmek,\n\nd) Sınavlarda tehditle kopya çekmek, kopya çeken öğrencilerin sınav salonundan çıkarılmasına engel olmak, kendi yerine başkasını sınava sokmak veya başkasının yerine sınava girmek.\n\n<|END|>\n<|END|>\n\n.\n\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.', source_nodes=[NodeWithScore(node=TextNode(id_='df368983-57b5-42c4-8252-44fdf1806afe', embedding=None, metadata={'page_label': '2', 'file_name': 'sinavkurallari-ogrenci.pdf', 'file_path': '/content/data/sinavkurallari-ogrenci.pdf', 'file_type': 'application/pdf', 'file_size': 78132, 'creation_date': '2024-06-04', 'last_modified_date': '2024-06-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'cr

In [77]:
print("DÖKÜMAN:\n\n",response.source_nodes[0].text)
print("===="*40)
print("CEVAP:\n\n",response)

DÖKÜMAN:

 14. Öğrenciler, sınav görevlisinin tüm talimat ve uyarılarına uymak ile yükümlüdürler. Verilen talimatlara uymayan öğrencilerin bu durumu, disiplin soruşturmasına da dayanak olacak şekilde tutanakla tespit altına alınır.   15. Her öğrenci salonu terk edene kadar kendi kâğıdından sorumludur. Sınav süresince ve sınav kâğıdının teslimi sırasında her öğrenci kendi sınav kâğıdının güvenliğini sağlamalı, diğer öğrenciler tarafından görülmeyecek şekilde tutmalıdır.   16. Sınavını tamamlayarak sınav evrakını gözetmene teslim eden öğrencinin, sırasına dönmeden sınav salonundan çıkması gerekir. Sınav soruları veya verilmiş yanıtlar herhangi bir şekilde kopyalanarak (yazarak, fotoğraf çekilerek, vb.) sınav salonundan çıkartılamaz. Bu amaca yönelik yapılan işlemler, kopya girişimi olarak değerlendirilir.   17. Sınavın son beş dakikasında salonu terk etmek yasaktır. Bu sürenin sonunda, öğrenciler soruları cevaplamayı bırakmalıdır ve ayağa kalkmamalıdırlar. Sınav bitiminde gözetmenler sır

## Built-in Agent

In [1]:
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

wiki_spec = WikipediaToolSpec()
# Get the search wikipedia tool
tool = wiki_spec.to_tool_list()[1]

# Create the Agent with load/search tools
agent = OpenAIAgent.from_tools(
    LoadAndSearchToolSpec.from_defaults(tool).to_tool_list(), verbose=True
)

ModuleNotFoundError: No module named 'llama_index'